In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import seaborn as sns
from itertools import product
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import ttest_ind
import umap.umap_ as umap
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
from lifelines.plotting import add_at_risk_counts
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.metrics import RocCurveDisplay, roc_auc_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler
import GEOparse
import mygene


In [19]:
geo_expr = pd.read_csv("GSE126669_tpm_length_scaled_matrix_donato_2020.txt.gz",sep='\t')
geo_expr

,ensembl_gene_id,CD_162,CD_180,CD_190,CD_208,CD_222,CD_224,CD_229,CD_264,CD_571,...,CD_Br16_50,CD_Br16_53,CD_Br16_55,CD_Br16_57,CD_Br16_63,CD_Br16_65,CD_LM2_30,CD_LM2_31,CD_LM2_35,CD_LM2_40
0,ENSG00000000003,22.546031,132.815481,6.111938,19.755002,37.292698,61.248864,39.007481,11.155147,7.854800,...,0.470975,2.555647,43.860216,0.000000,0.346900,0.614822,3.006699,23.576021,18.778100,0.000000
1,ENSG00000000419,1.807444,69.541754,313.802780,131.376263,130.484214,213.623156,182.205273,95.745255,151.237406,...,134.956354,100.222327,131.467506,62.716564,111.559069,85.646258,191.702573,119.235695,134.902188,47.190465
2,ENSG00000000457,0.136921,4.239567,25.404347,0.428466,0.298270,0.161185,12.095601,0.322610,0.493373,...,2.507807,20.753448,0.316210,17.038233,5.292762,4.024177,0.691426,0.744508,4.800156,0.811731
3,ENSG00000000460,1.940348,0.947305,0.281863,19.410337,0.147122,58.578698,51.410254,2.518942,0.000000,...,45.068547,14.690810,0.000000,0.000000,1.349334,1.044041,62.956622,46.260869,0.724249,4.143511
4,ENSG00000000971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.234714,12.360545,...,0.000000,0.000000,0.000000,0.000000,0.797774,0.000000,9.632186,0.734259,6.930036,0.564277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19492,ENSG00000284505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19493,ENSG00000284512,0.000000,0.000000,0.606923,0.197495,0.286540,0.000000,0.000000,9.817088,0.487661,...,0.543677,0.000000,0.000000,0.858719,1.348078,0.545199,0.291840,3.110367,0.000000,0.397652
19494,ENSG00000284526,2.911055,5.361395,10.198455,5.259593,3.607834,3.452370,9.364368,2.341739,30.277304,...,4.801306,11.360028,4.130827,12.666283,1.613898,11.872371,29.601240,12.280896,4.724077,5.526046
19495,ENSG00000284554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
mg = mygene.MyGeneInfo()
geneSyms = mg.querymany(list(geo_expr['ensembl_gene_id'].values) , scopes='ensembl.gene', fields='symbol', species='human')
geneSyms = pd.DataFrame(geneSyms)
geneSyms=geneSyms.dropna(axis=0,how='any',subset=['symbol'])[['query','symbol']]
common_genes_list=pd.Series(list(set(geo_expr['ensembl_gene_id']).intersection(set(geneSyms['query']))))
geo_expr=geo_expr[geo_expr['ensembl_gene_id'].isin(common_genes_list)]
geo_expr.insert(0,'geneSyms',geneSyms['symbol'].values)
geo_expr=geo_expr.drop(['ensembl_gene_id'],axis=1).set_index('geneSyms')
unwanted_integrins=['ITGB1BP1','ITGB3BP','ITGB1BP2','ITGB1P1']
geo_expr=geo_expr.loc[~(geo_expr.index.isin(unwanted_integrins))]
itg_expr=geo_expr.loc[geo_expr.index.str.contains('ITG')].index


querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-19497...done.
Finished.
113 input query terms found no hit:
	['ENSG00000069712', 'ENSG00000112096', 'ENSG00000116957', 'ENSG00000130489', 'ENSG00000130723', 'ENS
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [21]:
gse = GEOparse.get_GEO(geo="GSE126669", destdir="./")
geo_clin_matrix=gse.phenotype_data.copy()
geo_clin_matrix=geo_clin_matrix.assign(title=[sampleid.replace(' ','_') for sampleid in geo_clin_matrix['title']])


21-Feb-2023 14:52:55 DEBUG utils - Directory ./ already exists. Skipping.
21-Feb-2023 14:52:55 INFO GEOparse - File already exist: using local version.
21-Feb-2023 14:52:55 INFO GEOparse - Parsing ./GSE126669_family.soft.gz: 
21-Feb-2023 14:52:55 DEBUG GEOparse - DATABASE: GeoMiame
21-Feb-2023 14:52:55 DEBUG GEOparse - SERIES: GSE126669
21-Feb-2023 14:52:55 DEBUG GEOparse - PLATFORM: GPL18573
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3610995
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3610996
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3610997
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3610998
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3610999
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3611000
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3611001
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3611002
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3611003
21-Feb-2023 14:52:55 DEBUG GEOparse - SAMPLE: GSM3611004
21-Feb-2023 14:52:55 DEBUG GEOpars

In [22]:

for j,k in zip(geo_expr.columns,geo_clin_matrix['title']):
    if j!=k:
        print('problem',j,k)
        a+=1
    #else: 
    #print('the sample title list matches between geo_clin_matrix and geo_expr')

In [23]:
geo_clin_matrix[geo_clin_matrix.columns[geo_clin_matrix.columns.str.contains('characteristics')]]

,characteristics_ch1.0.model,characteristics_ch1.1.cell type,characteristics_ch1.2.hypoxia_red,characteristics_ch1.3.hypoxia_score,characteristics_ch1.4.hypoxia
GSM3610995,Br61,Circulating tumor cells (CTCs),Negative,Negative,Negative
GSM3610996,Br61,Circulating tumor cells (CTCs),Positive,Positive,Positive
GSM3610997,Br16,Circulating tumor cells (CTCs),Positive,Positive,Positive
GSM3610998,Br16,Circulating tumor cells (CTCs),Positive,Positive,Positive
GSM3610999,Br16,Circulating tumor cells (CTCs),Negative,Negative,Negative
GSM3611000,Br16,Circulating tumor cells (CTCs),Negative,Negative,Negative
GSM3611001,Br16,Circulating tumor cells (CTCs),Positive,Positive,Positive
GSM3611002,Br16,Circulating tumor cells (CTCs),Positive,Positive,Positive
GSM3611003,LM2,Circulating tumor cells (CTCs),Negative,Negative,Negative
GSM3611004,LM2,Circulating tumor cells (CTCs),Positive,Positive,Positive


In [24]:
geo_clin_matrix['characteristics_ch1.4.hypoxia'].value_counts()

Negative    17
Positive    14
Name: characteristics_ch1.4.hypoxia, dtype: int64

In [ ]:
normox_ids=geo_clin_matrix[geo_clin_matrix['characteristics_ch1.4.hypoxia']=='Negative']['title']
hypox_ids=geo_clin_matrix[geo_clin_matrix['characteristics_ch1.4.hypoxia']=='Positive']['title']
#one gene missing from donato list see other jupyter notebook
donato_upregulated = ['VEGFA','UBALD1','CLCN2','ZNF771','FAM13A','ABCC5','IKZF2','ZBTB43','IRS1',
                       'BIVM','PIK3C2B','LRIG2','MLLT3','PLD6','P4HA1','GDF15','LIPT1','MACC1','PPP1R3E',
                       'EPHX2','POFUT1','NIPAL1','CCNG2','HIF1A','CTSS']
donato_downregulated = ['TMEM125','VPS13D','POLR2J2','CCDC9','PGGT1B','SNAPC3']
donato_sig_genes= donato_upregulated + donato_downregulated

In [ ]:
geo_expr.loc[donato_sig_genes]

In [ ]:
#class weight balanced here
#make sure to remove random_state from the train-test split to 'randomforestclassifier'
print('this is random forest, using donato CTC signature gene list 31 not 32, distinguishing normoxic vs hypoxic CTC')
fig, ax = plt.subplots(1,2, figsize=(20, 8))
n=500

feature_importance=[]
accuracy=[]
roc_auc=[]
test_size=0.50
feature_importances={"feature": [],"importance": []}
for iteration in range(n):
    X = geo_expr.loc[donato_sig_genes].T
    y = geo_clin_matrix['characteristics_ch1.4.hypoxia']
    #scaler = StandardScaler()
    #X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) #stratify=y have error
    ### Random Forest Classifier ###
    classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight= 'balanced')
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    accuracy.append(accuracy_score(y_test,y_pred))
    #roc_auc.append(roc_auc_score(y_test, y_pred))
    for i,j in zip(X.columns,classifier.feature_importances_):
        feature_importances['feature'].append(i) 
        feature_importances['importance'].append(j)
#now plotting
pd.DataFrame(accuracy,columns=['accuracy']).plot(kind='hist',edgecolor='black',ax=ax[0])
ax[0].set_title(f"n= {n}",fontsize=25)
#ax[0].legend(fontsize=13)
plt.setp(ax[0].get_xticklabels(),fontsize=12)
plt.setp(ax[0].get_yticklabels(),fontsize=12)
ax[0].set_ylabel('Frequency',fontsize=25)
feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
feature_importance_df = feature_importance_df.sort_values(by='importance')
for i in ['top', 'left', 'right', 'bottom']:
    ax[0].spines[i].set_linewidth(1.6)

ax[0].minorticks_on()
ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
ax[0].set_xlabel('Accuracy',fontsize=25)


feature_importance_df_mean=feature_importance_df.groupby('feature').mean().sort_values(by='importance')
feature_importance_df_err=feature_importance_df.groupby('feature').std().loc[feature_importance_df_mean.index]
feature_importance_df_mean.plot(kind='bar',yerr=feature_importance_df_err,capsize=3,ax=ax[1])
ax[1].set_title(f"n= {n}",fontsize=25)
plt.setp(ax[1].get_xticklabels(),fontsize=12)
plt.setp(ax[1].get_yticklabels(),fontsize=12)
ax[1].set_xlabel('Feature',fontsize=25)
ax[1].legend(fontsize=13)

for i in ['top', 'left', 'right', 'bottom']:
    ax[1].spines[i].set_linewidth(1.6)

ax[1].minorticks_on()
ax[1].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[1].tick_params(axis='both', which='minor', width = 1.2, size = 5)    


#pd.DataFrame(roc_auc,columns=['roc_auc']).plot(kind='hist',edgecolor='black')
# plt.title(f"n= {n}",fontsize=25)
# #ax[0].legend(fontsize=13)
# plt.setp(ax[0].get_xticklabels(),fontsize=12)
# plt.setp(ax[0].get_yticklabels(),fontsize=12)
# ax[0].set_ylabel('Frequency',fontsize=25)
# feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
# feature_importance_df = feature_importance_df.sort_values(by='importance')
# for i in ['top', 'left', 'right', 'bottom']:
#     ax[0].spines[i].set_linewidth(1.6)

# ax[0].minorticks_on()
# ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
# ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
# ax[0].set_xlabel('Accuracy',fontsize=25)

plt.tight_layout()

#plt.savefig('figures/tcga_multiclass_rf_classifier_class-weight_bal.jpg', dpi = 1000, bbox_inches='tight')
#del temp_info_sample, temp_aur_bcm_df


In [ ]:
remove_bottom3 =[gene for gene in donato_sig_genes if gene not in ['MACC1','TMEM125','BIVM']]

In [ ]:
#class weight balanced here
#make sure to remove random_state from the train-test split to 'randomforestclassifier'
print('this is random forest, using only integrin expression, distinguishing normoxic vs hypoxic CTC')
fig, ax = plt.subplots(1,2, figsize=(20, 8))
n=500

feature_importance=[]
accuracy=[]
roc_auc=[]
test_size=0.50
feature_importances={"feature": [],"importance": []}
for iteration in range(n):
    X = geo_expr.loc[geo_expr.index.str.contains('ITG')].T
    y = geo_clin_matrix['characteristics_ch1.4.hypoxia']
    #scaler = StandardScaler()
    #X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) #stratify=y have error
    ### Random Forest Classifier ###
    classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight= 'balanced')
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    accuracy.append(accuracy_score(y_test,y_pred))
    #roc_auc.append(roc_auc_score(y_test, y_pred))
    for i,j in zip(X.columns,classifier.feature_importances_):
        feature_importances['feature'].append(i) 
        feature_importances['importance'].append(j)
#now plotting
pd.DataFrame(accuracy,columns=['accuracy']).plot(kind='hist',edgecolor='black',ax=ax[0])
ax[0].set_title(f"n= {n}",fontsize=25)
#ax[0].legend(fontsize=13)
plt.setp(ax[0].get_xticklabels(),fontsize=12)
plt.setp(ax[0].get_yticklabels(),fontsize=12)
ax[0].set_ylabel('Frequency',fontsize=25)
feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
feature_importance_df = feature_importance_df.sort_values(by='importance')
for i in ['top', 'left', 'right', 'bottom']:
    ax[0].spines[i].set_linewidth(1.6)

ax[0].minorticks_on()
ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
ax[0].set_xlabel('Accuracy',fontsize=25)


feature_importance_df_mean=feature_importance_df.groupby('feature').mean().sort_values(by='importance')
feature_importance_df_err=feature_importance_df.groupby('feature').std().loc[feature_importance_df_mean.index]
feature_importance_df_mean.plot(kind='bar',yerr=feature_importance_df_err,capsize=3,ax=ax[1])
ax[1].set_title(f"n= {n}",fontsize=25)
plt.setp(ax[1].get_xticklabels(),fontsize=12)
plt.setp(ax[1].get_yticklabels(),fontsize=12)
ax[1].set_xlabel('Feature',fontsize=25)
ax[1].legend(fontsize=13)

for i in ['top', 'left', 'right', 'bottom']:
    ax[1].spines[i].set_linewidth(1.6)

ax[1].minorticks_on()
ax[1].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[1].tick_params(axis='both', which='minor', width = 1.2, size = 5)    


#pd.DataFrame(roc_auc,columns=['roc_auc']).plot(kind='hist',edgecolor='black')
# plt.title(f"n= {n}",fontsize=25)
# #ax[0].legend(fontsize=13)
# plt.setp(ax[0].get_xticklabels(),fontsize=12)
# plt.setp(ax[0].get_yticklabels(),fontsize=12)
# ax[0].set_ylabel('Frequency',fontsize=25)
# feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
# feature_importance_df = feature_importance_df.sort_values(by='importance')
# for i in ['top', 'left', 'right', 'bottom']:
#     ax[0].spines[i].set_linewidth(1.6)

# ax[0].minorticks_on()
# ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
# ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
# ax[0].set_xlabel('Accuracy',fontsize=25)

plt.tight_layout()

#plt.savefig('figures/tcga_multiclass_rf_classifier_class-weight_bal.jpg', dpi = 1000, bbox_inches='tight')
#del temp_info_sample, temp_aur_bcm_df


In [ ]:
#class weight balanced here
#make sure to remove random_state from the train-test split to 'randomforestclassifier'
print('this is random forest, using top 3 integrin expression, distinguishing normoxic vs hypoxic CTC')
fig, ax = plt.subplots(1,2, figsize=(20, 8))
n=500

feature_importance=[]
accuracy=[]
roc_auc=[]
test_size=0.50
feature_importances={"feature": [],"importance": []}
for iteration in range(n):
    X = geo_expr.loc[['ITGA11','ITGB8',"ITGAE"]].T
    y = geo_clin_matrix['characteristics_ch1.4.hypoxia']
    #scaler = StandardScaler()
    #X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) #stratify=y have error
    ### Random Forest Classifier ###
    classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight= 'balanced')
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    accuracy.append(accuracy_score(y_test,y_pred))
    #roc_auc.append(roc_auc_score(y_test, y_pred))
    for i,j in zip(X.columns,classifier.feature_importances_):
        feature_importances['feature'].append(i) 
        feature_importances['importance'].append(j)
#now plotting
pd.DataFrame(accuracy,columns=['accuracy']).plot(kind='hist',edgecolor='black',ax=ax[0])
ax[0].set_title(f"n= {n}",fontsize=25)
#ax[0].legend(fontsize=13)
plt.setp(ax[0].get_xticklabels(),fontsize=12)
plt.setp(ax[0].get_yticklabels(),fontsize=12)
ax[0].set_ylabel('Frequency',fontsize=25)
feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
feature_importance_df = feature_importance_df.sort_values(by='importance')
for i in ['top', 'left', 'right', 'bottom']:
    ax[0].spines[i].set_linewidth(1.6)

ax[0].minorticks_on()
ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
ax[0].set_xlabel('Accuracy',fontsize=25)


feature_importance_df_mean=feature_importance_df.groupby('feature').mean().sort_values(by='importance')
feature_importance_df_err=feature_importance_df.groupby('feature').std().loc[feature_importance_df_mean.index]
feature_importance_df_mean.plot(kind='bar',yerr=feature_importance_df_err,capsize=3,ax=ax[1])
ax[1].set_title(f"n= {n}",fontsize=25)
plt.setp(ax[1].get_xticklabels(),fontsize=12)
plt.setp(ax[1].get_yticklabels(),fontsize=12)
ax[1].set_xlabel('Feature',fontsize=25)
ax[1].legend(fontsize=13)

for i in ['top', 'left', 'right', 'bottom']:
    ax[1].spines[i].set_linewidth(1.6)

ax[1].minorticks_on()
ax[1].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[1].tick_params(axis='both', which='minor', width = 1.2, size = 5)    


#pd.DataFrame(roc_auc,columns=['roc_auc']).plot(kind='hist',edgecolor='black')
# plt.title(f"n= {n}",fontsize=25)
# #ax[0].legend(fontsize=13)
# plt.setp(ax[0].get_xticklabels(),fontsize=12)
# plt.setp(ax[0].get_yticklabels(),fontsize=12)
# ax[0].set_ylabel('Frequency',fontsize=25)
# feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
# feature_importance_df = feature_importance_df.sort_values(by='importance')
# for i in ['top', 'left', 'right', 'bottom']:
#     ax[0].spines[i].set_linewidth(1.6)

# ax[0].minorticks_on()
# ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
# ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
# ax[0].set_xlabel('Accuracy',fontsize=25)

plt.tight_layout()

#plt.savefig('figures/tcga_multiclass_rf_classifier_class-weight_bal.jpg', dpi = 1000, bbox_inches='tight')
#del temp_info_sample, temp_aur_bcm_df


In [ ]:
#class weight balanced here
#make sure to remove random_state from the train-test split to 'randomforestclassifier'

fig, ax = plt.subplots(1,2, figsize=(20, 8))
n=500

feature_importance=[]
accuracy=[]
roc_auc=[]
test_size=0.50
feature_importances={"feature": [],"importance": []}
for iteration in range(n):
    X = geo_expr.loc[['ITGA11','ITGB8','ITGAE']+donato_sig_genes].T
    y = geo_clin_matrix['characteristics_ch1.4.hypoxia']
    #scaler = StandardScaler()
    #X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) #stratify=y have error
    ### Random Forest Classifier ###
    classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight= 'balanced')
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    accuracy.append(accuracy_score(y_test,y_pred))
    #roc_auc.append(roc_auc_score(y_test, y_pred))
    for i,j in zip(X.columns,classifier.feature_importances_):
        feature_importances['feature'].append(i) 
        feature_importances['importance'].append(j)
#now plotting
pd.DataFrame(accuracy,columns=['accuracy']).plot(kind='hist',edgecolor='black',ax=ax[0])
ax[0].set_title(f"n= {n}",fontsize=25)
#ax[0].legend(fontsize=13)
plt.setp(ax[0].get_xticklabels(),fontsize=12)
plt.setp(ax[0].get_yticklabels(),fontsize=12)
ax[0].set_ylabel('Frequency',fontsize=25)
feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
feature_importance_df = feature_importance_df.sort_values(by='importance')
for i in ['top', 'left', 'right', 'bottom']:
    ax[0].spines[i].set_linewidth(1.6)

ax[0].minorticks_on()
ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
ax[0].set_xlabel('Accuracy',fontsize=25)


feature_importance_df_mean=feature_importance_df.groupby('feature').mean().sort_values(by='importance')
feature_importance_df_err=feature_importance_df.groupby('feature').std().loc[feature_importance_df_mean.index]
feature_importance_df_mean.plot(kind='bar',yerr=feature_importance_df_err,capsize=3,ax=ax[1])
ax[1].set_title(f"n= {n}",fontsize=25)
plt.setp(ax[1].get_xticklabels(),fontsize=12)
plt.setp(ax[1].get_yticklabels(),fontsize=12)
ax[1].set_xlabel('Feature',fontsize=25)
ax[1].legend(fontsize=13)

for i in ['top', 'left', 'right', 'bottom']:
    ax[1].spines[i].set_linewidth(1.6)

ax[1].minorticks_on()
ax[1].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[1].tick_params(axis='both', which='minor', width = 1.2, size = 5)    


#pd.DataFrame(roc_auc,columns=['roc_auc']).plot(kind='hist',edgecolor='black')
# plt.title(f"n= {n}",fontsize=25)
# #ax[0].legend(fontsize=13)
# plt.setp(ax[0].get_xticklabels(),fontsize=12)
# plt.setp(ax[0].get_yticklabels(),fontsize=12)
# ax[0].set_ylabel('Frequency',fontsize=25)
# feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
# feature_importance_df = feature_importance_df.sort_values(by='importance')
# for i in ['top', 'left', 'right', 'bottom']:
#     ax[0].spines[i].set_linewidth(1.6)

# ax[0].minorticks_on()
# ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
# ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
# ax[0].set_xlabel('Accuracy',fontsize=25)

plt.tight_layout()

#plt.savefig('figures/tcga_multiclass_rf_classifier_class-weight_bal.jpg', dpi = 1000, bbox_inches='tight')
#del temp_info_sample, temp_aur_bcm_df


In [ ]:
#class weight balanced here
#make sure to remove random_state from the train-test split to 'randomforestclassifier'

fig, ax = plt.subplots(1,2, figsize=(20, 8))
n=500

feature_importance=[]
accuracy=[]
roc_auc=[]
test_size=0.50
feature_importances={"feature": [],"importance": []}
for iteration in range(n):
    X = geo_expr.loc[['ITGA11','ITGB8','ITGAE']+remove_bottom3].T
    y = geo_clin_matrix['characteristics_ch1.4.hypoxia']
    #scaler = StandardScaler()
    #X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) #stratify=y have error
    ### Random Forest Classifier ###
    classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight= 'balanced')
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    accuracy.append(accuracy_score(y_test,y_pred))
    #roc_auc.append(roc_auc_score(y_test, y_pred))
    for i,j in zip(X.columns,classifier.feature_importances_):
        feature_importances['feature'].append(i) 
        feature_importances['importance'].append(j)
#now plotting
pd.DataFrame(accuracy,columns=['accuracy']).plot(kind='hist',edgecolor='black',ax=ax[0])
ax[0].set_title(f"n= {n}",fontsize=25)
#ax[0].legend(fontsize=13)
plt.setp(ax[0].get_xticklabels(),fontsize=12)
plt.setp(ax[0].get_yticklabels(),fontsize=12)
ax[0].set_ylabel('Frequency',fontsize=25)
feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
feature_importance_df = feature_importance_df.sort_values(by='importance')
for i in ['top', 'left', 'right', 'bottom']:
    ax[0].spines[i].set_linewidth(1.6)

ax[0].minorticks_on()
ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
ax[0].set_xlabel('Accuracy',fontsize=25)


feature_importance_df_mean=feature_importance_df.groupby('feature').mean().sort_values(by='importance')
feature_importance_df_err=feature_importance_df.groupby('feature').std().loc[feature_importance_df_mean.index]
feature_importance_df_mean.plot(kind='bar',yerr=feature_importance_df_err,capsize=3,ax=ax[1])
ax[1].set_title(f"n= {n}",fontsize=25)
plt.setp(ax[1].get_xticklabels(),fontsize=12)
plt.setp(ax[1].get_yticklabels(),fontsize=12)
ax[1].set_xlabel('Feature',fontsize=25)
ax[1].legend(fontsize=13)

for i in ['top', 'left', 'right', 'bottom']:
    ax[1].spines[i].set_linewidth(1.6)

ax[1].minorticks_on()
ax[1].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
ax[1].tick_params(axis='both', which='minor', width = 1.2, size = 5)    


#pd.DataFrame(roc_auc,columns=['roc_auc']).plot(kind='hist',edgecolor='black')
# plt.title(f"n= {n}",fontsize=25)
# #ax[0].legend(fontsize=13)
# plt.setp(ax[0].get_xticklabels(),fontsize=12)
# plt.setp(ax[0].get_yticklabels(),fontsize=12)
# ax[0].set_ylabel('Frequency',fontsize=25)
# feature_importance_df=pd.DataFrame(feature_importances).set_index('feature')
# feature_importance_df = feature_importance_df.sort_values(by='importance')
# for i in ['top', 'left', 'right', 'bottom']:
#     ax[0].spines[i].set_linewidth(1.6)

# ax[0].minorticks_on()
# ax[0].tick_params(axis='both', which='major', labelsize=20, width = 1.7, size = 8,pad=10)
# ax[0].tick_params(axis='both', which='minor', width = 1.2, size = 5)   
# ax[0].set_xlabel('Accuracy',fontsize=25)

plt.tight_layout()

#plt.savefig('figures/tcga_multiclass_rf_classifier_class-weight_bal.jpg', dpi = 1000, bbox_inches='tight')
#del temp_info_sample, temp_aur_bcm_df


In [ ]:
for chosen_gene in itg_expr:
    fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
    max_lim=geo_expr.loc[chosen_gene].max()
    min_lim=geo_expr.loc[chosen_gene].min()    

    sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,inner='point',
                   x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
                   color='blue',
                           scale='width',ax=ax[0])
    ax[0].set_title(chosen_gene)
    ax[0].set_ylim(min_lim,max_lim)
    sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,inner='point',
                   x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
                   color='red',
                           scale='width',ax=ax[1])
    ax[1].set_title(chosen_gene)
    ax[1].set_ylim(min_lim,max_lim)

In [ ]:
for chosen_gene in donato_sig_genes:
    fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
    max_lim=geo_expr.loc[chosen_gene].max()
    min_lim=geo_expr.loc[chosen_gene].min()    

    sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,inner='point',
                   x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
                   color='blue',
                           scale='width',ax=ax[0])
    ax[0].set_title(chosen_gene)
    ax[0].set_ylim(min_lim,max_lim)
    sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,inner='point',
                   x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
                   color='red',
                           scale='width',ax=ax[1])
    ax[1].set_title(chosen_gene)
    ax[1].set_ylim(min_lim,max_lim)

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
chosen_gene='ITGA11'

sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,inner='points',
               x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
               color='blue',
                       scale='width',ax=ax[0])
ax[0].set_title(chosen_gene)
ax[0].set_ylim(-0.1,2.5)
sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,inner='points',
               x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
               color='red',
                       scale='width',ax=ax[1])
ax[1].set_title(chosen_gene)
ax[1].set_ylim(-0.1,2.5)

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
chosen_gene='ITGB1'
max_lim=geo_expr.loc[chosen_gene].max()
min_lim=geo_expr.loc[chosen_gene].min()
sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
               color='blue',
                       scale='width',ax=ax[0])
ax[0].set_title(chosen_gene)
ax[0].set_ylim(min_lim,max_lim)
sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
               color='red',
                       scale='width',ax=ax[1])
ax[1].set_title(chosen_gene)
ax[1].set_ylim(min_lim,max_lim)

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
chosen_gene='ITGB1'

sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
               color='blue',
                       scale='width',ax=ax[0])
ax[0].set_title(chosen_gene)
ax[0].set_ylim(-60,350)
sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
               color='red',
                       scale='width',ax=ax[1])
ax[1].set_title(chosen_gene)
ax[1].set_ylim(-60,350)

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
chosen_gene='VEGFA'

sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
               color='blue',
                       scale='width',ax=ax[0])
ax[0].set_title(chosen_gene)
ax[0].set_ylim(-30,100)
sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
               color='red',
                       scale='width',ax=ax[1])
ax[1].set_title(chosen_gene)
ax[1].set_ylim(-30,100)

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
chosen_gene='IKZF2'

sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
               color='blue',
                       scale='width',ax=ax[0])
ax[0].set_title(chosen_gene)
ax[0].set_ylim(-10,50)
sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
               color='red',
                       scale='width',ax=ax[1])
ax[1].set_title(chosen_gene)
ax[1].set_ylim(-10,50)

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(10,7))
chosen_gene='PGGT1B'

sns.violinplot(y=geo_expr[normox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(normox_ids)]['characteristics_ch1.4.hypoxia'],
               color='blue',
                       scale='width',ax=ax[0])
ax[0].set_title(chosen_gene)
ax[0].set_ylim(-50,200)
sns.violinplot(y=geo_expr[hypox_ids].loc[chosen_gene].values,
               x=geo_clin_matrix[geo_clin_matrix.title.isin(hypox_ids)]['characteristics_ch1.4.hypoxia'],
               color='red',
                       scale='width',ax=ax[1])
ax[1].set_title(chosen_gene)
ax[1].set_ylim(-50,200)

In [ ]:
#looking at some correlations
#only normoxia
normoxic_pairwise_corr_geo_expr = pd.DataFrame(np.corrcoef(geo_expr[normox_ids].T,
                                                          rowvar=False),
                                          columns=geo_expr[normox_ids].T.columns,
                                              index=geo_expr[normox_ids].T.columns)
#only hypoxia
hypoxic_pairwise_corr_geo_expr = pd.DataFrame(np.corrcoef(geo_expr[hypox_ids].T,
                                                          rowvar=False),
                                          columns=geo_expr[hypox_ids].T.columns,
                                              index=geo_expr[hypox_ids].T.columns)




In [ ]:
corr_cutoff=0.5
chosen_gene='HIF1A'
normoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr][normoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr]>=corr_cutoff]


In [ ]:
corr_cutoff=0.5
chosen_gene='HIF1A'
hypoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr][hypoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr]>=corr_cutoff]


In [ ]:
corr_cutoff=0.5
chosen_gene='PGGT1B'
normoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr][normoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr]>=corr_cutoff]


In [ ]:
corr_cutoff=0.5
chosen_gene='PGGT1B'
hypoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr][hypoxic_pairwise_corr_geo_expr[chosen_gene][itg_expr]>=corr_cutoff]


In [ ]:
#in normoxic sample ITGA11 is correlated with which of the 32 signature genes?
corr_cutoff=0.5
chosen_gene='ITGA11'
normoxic_pairwise_corr_geo_expr[chosen_gene][donato_sig_genes][normoxic_pairwise_corr_geo_expr[chosen_gene][donato_sig_genes]>=corr_cutoff]


In [ ]:
#in hypoxic sample ITGA11 is correlated with which of the 32 signature genes?
corr_cutoff=0.5
chosen_gene='ITGA11'
hypoxic_pairwise_corr_geo_expr[chosen_gene][donato_sig_genes][hypoxic_pairwise_corr_geo_expr[chosen_gene][donato_sig_genes]>=corr_cutoff]


In [ ]:
donato_sig_genes